# PyCity Schools Analysis

## Your analysis here
  ## My initial observations are as follows:
  
  #There were 15 unique schools in the combined data set. 
  #There were 39,170 students within the 15 unique schools.
  #The total budget for the schools was just under $24.7 million dollars. 
  #The percentage of students passing reading was just under 75 percent.
  #The percentage of students passing math was just under 86 percent.
  #The combinded percentage of students passing both math and reading was just over 65 percent.
---

In [23]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [24]:
school_shape = school_data.shape
print(school_shape)
print(school_data.head(15))
student_shape = student_data.shape
print("student data")
print(student_shape)
print(student_data.head(15))

(15, 5)
    School ID            school_name      type  size   budget
0           0      Huang High School  District  2917  1910635
1           1   Figueroa High School  District  2949  1884411
2           2    Shelton High School   Charter  1761  1056600
3           3  Hernandez High School  District  4635  3022020
4           4    Griffin High School   Charter  1468   917500
5           5     Wilson High School   Charter  2283  1319574
6           6    Cabrera High School   Charter  1858  1081356
7           7     Bailey High School  District  4976  3124928
8           8     Holden High School   Charter   427   248087
9           9       Pena High School   Charter   962   585858
10         10     Wright High School   Charter  1800  1049400
11         11  Rodriguez High School  District  3999  2547363
12         12    Johnson High School  District  4761  3094650
13         13       Ford High School  District  2739  1763916
14         14     Thomas High School   Charter  1635  1043130


## District Summary

In [25]:
# Calculate the total number of unique schools
school_count = school_data["school_name"].nunique()
print("school_count", school_count)

school_count 15


In [26]:
# Calculate the total number of students
student_count = student_data["Student ID"].nunique()
print("student_count", student_count)

student_count 39170


In [27]:
# Calculate the total budget
total_budget = school_data["budget"].sum()
print("total_budget", total_budget)

total_budget 24649428


In [28]:
# Calculate the average (mean) math score
average_math_score = student_data["math_score"].mean()
print("average_math_score", average_math_score)

average_math_score 78.98537145774827


In [29]:
# Calculate the average (mean) reading score
average_reading_score = student_data["reading_score"].mean()
print("average_reading_score", average_reading_score)

average_reading_score 81.87784018381414


In [30]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [31]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [32]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
count = 0 
for score in student_data["reading_score"]:
    if score >= 70:
        count = count + 1

passing_reading_count = count
passing_reading_percentage = (passing_reading_count / student_count) * 100
print("passing_reading_percentage", passing_reading_percentage)

passing_reading_percentage 85.80546336482001


In [33]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({
    "Total number of unique schools" : [school_count],
    "Total Students" : [student_count],
    "Total Budget"   : [total_budget],
    "Average Math Score" : [average_math_score],
    "Average Reading Score" : [average_reading_score],
    "% Passing Math" : [passing_math_percentage],
    "% Passing Reading" : [passing_reading_percentage],
    "% Overall Passing" : [overall_passing_rate],
})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total number of unique schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [34]:
# Use the code provided to select all of the school types
group_school_student_df = school_data_complete.groupby(["school_name"])
school_type = group_school_student_df["type"].first()

In [35]:
# Calculate the total student count per school
per_school_counts = group_school_student_df["Student ID"].count()
per_school_counts.head()

school_name
Bailey High School      4976
Cabrera High School     1858
Figueroa High School    2949
Ford High School        2739
Griffin High School     1468
Name: Student ID, dtype: int64

In [36]:
# Calculate the total school budget and per capita spending per school
per_school_budget = group_school_student_df["budget"].first()
per_school_capita = per_school_budget / total_students

In [37]:
# Calculate the average test scores per school
per_school_math = group_school_student_df["math_score"].mean()
per_school_reading = group_school_student_df["reading_score"].mean()

In [38]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby(["school_name"])["math_score"].count()
school_students_passing_math = (students_passing_math / total_students) * 100

In [39]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70].groupby(["school_name"])["reading_score"].count()
school_students_passing_reading = (students_passing_reading / total_students) * 100

In [40]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()

In [41]:
# Use the provided code to calculate the passing rates

per_school_passing_math = (school_students_passing_math / per_school_counts) * 100
per_school_passing_reading = (school_students_passing_reading / per_school_counts) * 100
overall_passing_rate = (school_students_passing_math_and_reading / per_school_counts) * 100

In [42]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({
    "School Name" : [school_name],
    "School Type" : [school_type],
    "School Counts"   : [per_school_counts],
    "School Budget" : [per_school_budget],
    "School Capita" : [per_school_capita],
    "School Math Score" : [per_school_math],
    "School Reading Score" : [per_school_reading],
    "Students Passing Math" : [students_passing_math],
    "Percentage of Students Passing Math" : [school_students_passing_math],
    "Students Passing Reading" : [students_passing_reading],
    "Percentage of Students Passing Reading" : [school_students_passing_reading],
    "Students Passing Math and Reading" : [students_passing_math_and_reading],
    "Students Passing Math and Reading by School" : [school_students_passing_math_and_reading],
    "Schools Passing Math" : [per_school_passing_math],
    "Schools Passing Reading" : [per_school_passing_reading],
    "Overall Passing Reading" : [overall_passing_rate],   
})

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

SyntaxError: invalid syntax (2536131374.py, line 2)

## Highest-Performing Schools (by % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(["overall_passing_rate"], ascending = false)

top_schools.head(5)

## Bottom Performing Schools (By % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(["overall_passing_rate"], ascending=True)
bottom_schools.head(5)

## Math Scores by Grade

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = school_data.loc[school_stud_df["grade"] == "9th"].groupby(["school_name"])["math_score"].mean()
tenth_grader_math_scores = school_data.loc[school_stud_df["grade"] == "10th"].groupby(["school_name"])["math_score"].mean()
eleventh_grader_math_scores = school_data.loc[school_stud_df["grade"] == "11th"].groupby(["school_name"])["math_score"].mean()
twelfth_grader_math_scores = school_data.loc[school_stud_df["grade"] == "12th"].groupby(["school_name"])["math_score"].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({"9th": ninth_grade_math_scores,
      "10th": tenth_grader_math_scores,
      "11th": eleventh_grader_math_scores,
      "12th": twelfth_grader_math_scores})

# Minor data wrangling
math_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

## Reading Score by Grade 

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = school_data.loc[school_stud_df["grade"] == "9th"].groupby(["school_name"])["reading_score"].mean()
tenth_grader_reading_scores = school_data.loc[school_stud_df["grade"] == "10th"].groupby(["school_name"])["reading_score"].mean()
eleventh_grader_reading_scores = school_data.loc[school_stud_df["grade"] == "11th"].groupby(["school_name"])["reading_score"].mean()
twelfth_grader_reading_scores = school_data.loc[school_stud_df["grade"] == "12th"].groupby(["school_name"])["reading_score"].mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({"9th": ninth_grade_reading_scores,
      "10th": tenth_grader_reading_scores,
      "11th": eleventh_grader_reading_scores,
      "12th": twelfth_grader_reading_scores})

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

## Scores by School Spending

In [ ]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [ ]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [ ]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = 
school_spending_df

In [ ]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [ ]:
# Assemble into DataFrame
spending_summary = 

# Display results
spending_summary

## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = 
per_school_summary

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = 

# Display results
size_summary

## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = 

# Display results
type_summary